# PATE GAN based on DCGAN

We will attempt to employ sophisticated tecniques of PATE GANs

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np
%load_ext autoreload
%autoreload 2

In [42]:
n_teachers = 10

In [2]:
import os

GDRIVE_PATH = '/data/apsidorenko'
THIS_EXERCISE_PATH = os.path.join(GDRIVE_PATH, "GAN_exercises")
MODELS_HOME = os.path.join(THIS_EXERCISE_PATH, "mnist guns")

In [3]:
GENERATOR_FILE = os.path.join(MODELS_HOME, 'generator_pate.pt')
DISCRIMINATOR_FILE = os.path.join(MODELS_HOME, 'student_pate.pt')
#TODO teacher's files - ?

In [4]:
os.makedirs(THIS_EXERCISE_PATH, exist_ok=True)
os.makedirs(MODELS_HOME, exist_ok=True)

In [5]:
class Reshape(torch.nn.Module):
  """
  Reshapes a tensor starting from the 1st dimension (not 0th),
  i. e. without influencing the batch dimension.
  """
  def __init__(self, *shape):
    super(Reshape, self).__init__()
    self.shape = shape

  def forward(self, x):
    return x.view(x.shape[0], *self.shape)
  
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.shape[0], -1)

# Load data

In [11]:
LEAK_SHARE = 0.2 #how much elements have been delivered to an adversary

In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('https://query.data.world/s/nap7jvxtupud25z5ljvtbzzjjsqqay')
df.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_774,column_775,column_776,column_777,column_778,column_779,column_780,column_781,column_782,column_783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
target = pd.read_csv('https://query.data.world/s/sn3dximsq5sw3a6wtqoc3okulevugz')
target.head()

,column_0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [35]:
from sklearn.model_selection import train_test_split
train, test, tar_train, tar_test = train_test_split(df, target, test_size=0.2, random_state=12345)

In [36]:
train = np.array(train, dtype='float')
test = np.array(test, dtype='float')
tar_train = np.array(tar_train, dtype='float')
tar_test = np.array(tar_test, dtype='float')

train= train.reshape((-1, 1, 28, 28)) / 255.
test= test.reshape((-1, 1, 28, 28)) / 255.

In [37]:
n_leak = round(LEAK_SHARE * train.shape[0])
train_leak = train[np.random.permutation(train.shape[0])]
train_check = train_leak[n_leak:]
train_leak = train_leak[0:n_leak]

# The net itself

Generator (the only one) with a discriminator as the student.

In [44]:
CODE_SIZE = 100
DROPOUT_RATE = 0.2
nc=1
nz=100
ngf=64
ndf=64

try:
  generator = torch.load(GENERATOR_FILE)
  discriminator = torch.load(DISCRIMINATOR_FILE)
except FileNotFoundError:
  print('Files have not been not found: making new nets\n')
  generator = torch.nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf,nc, kernel_size=1, stride=1, padding=2, bias=False),
            nn.Tanh()
        ).cuda()

  student = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, 1, 4, 2, 1, bias=False),
            #nn.Sigmoid()
        ).cuda()

Files have not been not found: making new nets



Several teachers.

In [62]:
teacher = []

In [68]:
for i in range(n_teachers):
    teacher.append(nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, 1, 4, 2, 1, bias=False),
            #nn.Sigmoid()
        ).cuda())

In [47]:
#pred is teacher's prediction: 2d array: pred[i][0] - likelihood of fake, pred[i][1] = 1 - pred[i][0] - likelihood of real
def PATE(lam, pred):
    return np.argmax(pred + np.random.exponential(1/lam, pred.shape[1]) - np.random.exponential(1/lam, pred.shape[1]))

In [48]:
EPSILON = 0.01
lam = train.shape[0] / EPSILON #see page 4, there is a theorem

In [49]:
def sample_fake(batch_size):
  noise = torch.randn(batch_size, CODE_SIZE, 1, 1, device="cuda")
  return generator(noise)

### Train set division

In [50]:
train_teach = train.reshape((10, -1, 1, 28, 28)) #train set division

In [51]:
train.shape

(56000, 1, 28, 28)

In [52]:
train_teach.shape

(10, 5600, 1, 28, 28)

In [53]:
def sample_images_for_teacher(batch_size, num):
    ids = np.random.choice(len(train_teach[num], size=batch_size))
    return torch.tensor(train_teach[num, ids], device="cuda").float()

In [54]:
def sample_images(batch_size, train=train):
  ids = np.random.choice(len(train), size=batch_size)
  return torch.tensor(train[ids], device="cuda").float()

### Losses (?) 

In [72]:

def generator_loss(fake):
    return logsigmoid(-student(
              fake
          )).mean() #log(1 - sigmoid(student_prediction))
  
def student_loss(real, fake, pred):
    return -logsigmoid(discriminator(
              real 
          )).mean() - \
          logsigmoid(-discriminator(
              fake
          )).mean()

def teacher_loss(real, fake, num):
    return -logsigmoid(teacher[num](
              real
          )).mean() - \
          logsigmoid(-teacher[num](
              fake
          )).mean()



In [70]:
optimizer_generator = \
    torch.optim.RMSprop(generator.parameters(), lr=0.001)
optimizer_student = \
    torch.optim.RMSprop(student.parameters(), lr=0.001)

student_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=10, gamma=0.999)
gen_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_generator, step_size=10, gamma=0.999)
#TODO teacher optimisers - ???
optimizer_teacher = []
teacher_scheduler = []
for i in range(n_teachers):
    optimizer_teacher.append(torch.optim.RMSprop(teacher[i].parameters(), lr=0.001))
    teacher_scheduler.append(torch.optim.lr_scheduler.StepLR(optimizer_teacher[i], step_size=10, gamma=0.999))

In [71]:
VALIDATION_INTERVAL = 150
SAVE_INTERVAL = 500
DISCRIMINATOR_ITERATIONS_PER_GENEREATOR = 5
BATCH_SIZE=128

In [60]:
from IPython.display import clear_output
for i in range(2000):
  # Set our models to training mode:
  generator.train()
  student.train()
  gen_scheduler.step()
  student_scheduler.step()
  for i in range(n_teachers):
  
  # Several discriminator updates per step:
  for j in range(DISCRIMINATOR_ITERATIONS_PER_GENEREATOR):
    # Sampling reals and fakes
    real = sample_images(BATCH_SIZE)
    fake = sample_fake(BATCH_SIZE)
    
    # Calculating the loss
    discriminator_loss_this_iter = student_loss(real, fake)
    
    # Doing our regular optimization step for the discriminator
    optimizer_discriminator.zero_grad()
    discriminator_loss_this_iter.backward()
    optimizer_discriminator.step()

  # Pass the discriminator loss to Tensorboard for plotting 
  #summary_writer.add_scalar("discriminator loss", discriminator_loss_this_iter,
   #                          global_step=i)

  # Now it's generator's time to learn:
  generator_loss_this_iter = generator_loss(sample_fake(BATCH_SIZE))
  #summary_writer.add_scalar("generator loss", generator_loss_this_iter,
   #                         global_step=i)
  optimizer_generator.zero_grad()
  generator_loss_this_iter.backward()
  optimizer_generator.step()

  if i % SAVE_INTERVAL == 0:
    torch.save(generator, GENERATOR_FILE)
    torch.save(discriminator, DISCRIMINATOR_FILE)
    
  if i % VALIDATION_INTERVAL == 0:
    clear_output(wait=True)
    generator.eval()
    imgs = sample_fake(25).cpu().detach().numpy()
    plot_images(imgs.clip(0, 1), title='Iteration '+str(i));
    plt.show();

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


NameError: name 'discriminator_loss' is not defined